# Dashboard Tương tác với Plotly - Bài 3: Biểu đồ Tương tác và Dashboard

Notebook này tạo dashboard tương tác với Plotly để phân tích dữ liệu một cách trực quan và sinh động.

## 🔧 Thiết lập môi trường

In [ ]:
# Cài đặt Plotly nếu chưa có
!pip install plotly kaleido

# Import thư viện
import sys
sys.path.append('/app')

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import database helper
from db_connection import get_db_connection

# Thiết lập Plotly để hiển thị trong notebook
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

print("✅ Đã thiết lập môi trường Plotly thành công!")

## 🔌 Kết nối và tải dữ liệu

In [ ]:
# Kết nối database
db = get_db_connection()
print("✅ Đã kết nối thành công đến SQL Server!")

# Tải dữ liệu tổng hợp cho dashboard
dashboard_query = """
SELECT 
    c.CustomerName,
    c.City,
    c.JoinDate,
    p.ProductName,
    p.Category,
    p.Price,
    o.OrderDate,
    od.Quantity,
    (od.Quantity * p.Price) as Revenue,
    o.OrderID,
    c.CustomerID,
    p.ProductID
FROM Customers c
JOIN Orders o ON c.CustomerID = o.CustomerID
JOIN OrderDetails od ON o.OrderID = od.OrderID
JOIN Products p ON od.ProductID = p.ProductID
ORDER BY o.OrderDate, c.CustomerName
"""

df = db.query_to_dataframe(dashboard_query)
print(f"📊 Đã tải {len(df)} records cho dashboard")
print("\n🔍 Mẫu dữ liệu:")
display(df.head())

## 📊 Dashboard Tổng quan Doanh thu

In [ ]:
# Tạo dashboard tổng quan với subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        '💰 Doanh thu theo thời gian',
        '🏙️ Doanh thu theo thành phố', 
        '📂 Doanh thu theo danh mục sản phẩm',
        '👥 Top khách hàng VIP'
    ),
    specs=[
        [{"type": "scatter"}, {"type": "bar"}],
        [{"type": "pie"}, {"type": "bar"}]
    ]
)

# 1. Doanh thu theo thời gian
daily_revenue = df.groupby('OrderDate')['Revenue'].sum().reset_index()
fig.add_trace(
    go.Scatter(
        x=daily_revenue['OrderDate'],
        y=daily_revenue['Revenue'],
        mode='lines+markers',
        name='Doanh thu hàng ngày',
        line=dict(color='blue', width=3),
        marker=dict(size=8)
    ),
    row=1, col=1
)

# 2. Doanh thu theo thành phố
city_revenue = df.groupby('City')['Revenue'].sum().sort_values(ascending=True)
fig.add_trace(
    go.Bar(
        x=city_revenue.values,
        y=city_revenue.index,
        orientation='h',
        name='Doanh thu theo TP',
        marker_color='lightblue'
    ),
    row=1, col=2
)

# 3. Doanh thu theo danh mục
category_revenue = df.groupby('Category')['Revenue'].sum()
fig.add_trace(
    go.Pie(
        labels=category_revenue.index,
        values=category_revenue.values,
        name="Danh mục",
        hole=0.3
    ),
    row=2, col=1
)

# 4. Top khách hàng VIP
top_customers = df.groupby('CustomerName')['Revenue'].sum().nlargest(5)
fig.add_trace(
    go.Bar(
        x=top_customers.values,
        y=top_customers.index,
        orientation='h',
        name='Top khách hàng',
        marker_color='gold'
    ),
    row=2, col=2
)

# Cập nhật layout
fig.update_layout(
    height=800,
    showlegend=False,
    title_text="📊 DASHBOARD TỔNG QUAN DOANH NGHIỆP",
    title_x=0.5,
    title_font_size=20
)

# Cập nhật axes labels
fig.update_xaxes(title_text="Ngày", row=1, col=1)
fig.update_yaxes(title_text="Doanh thu ($)", row=1, col=1)
fig.update_xaxes(title_text="Doanh thu ($)", row=1, col=2)
fig.update_xaxes(title_text="Doanh thu ($)", row=2, col=2)

fig.show()

# Hiển thị thống kê tổng quan
total_revenue = df['Revenue'].sum()
total_orders = df['OrderID'].nunique()
total_customers = df['CustomerID'].nunique()
avg_order_value = df.groupby('OrderID')['Revenue'].sum().mean()

print(f"\n📈 THỐNG KÊ TỔNG QUAN:")
print(f"💰 Tổng doanh thu: ${total_revenue:,.2f}")
print(f"📦 Tổng số đơn hàng: {total_orders:,}")
print(f"👥 Tổng số khách hàng: {total_customers:,}")
print(f"💵 Giá trị trung bình đơn hàng: ${avg_order_value:.2f}")

## 🛍️ Phân tích Sản phẩm Tương tác

In [ ]:
# Tạo bubble chart cho phân tích sản phẩm
product_summary = df.groupby(['ProductName', 'Category', 'Price']).agg({
    'Quantity': 'sum',
    'Revenue': 'sum',
    'OrderID': 'nunique'
}).reset_index()
product_summary.columns = ['ProductName', 'Category', 'Price', 'TotalQuantity', 'TotalRevenue', 'OrderCount']

# Bubble chart: Giá vs Số lượng bán, size = Doanh thu
fig_bubble = px.scatter(
    product_summary, 
    x='Price', 
    y='TotalQuantity',
    size='TotalRevenue',
    color='Category',
    hover_name='ProductName',
    hover_data={'TotalRevenue': ':$,.0f', 'OrderCount': True},
    title='🛍️ Phân tích Sản phẩm: Giá vs Số lượng bán (Size = Doanh thu)',
    labels={
        'Price': 'Giá sản phẩm ($)',
        'TotalQuantity': 'Tổng số lượng bán',
        'Category': 'Danh mục'
    },
    size_max=60
)

fig_bubble.update_layout(
    height=600,
    title_x=0.5,
    title_font_size=16
)

fig_bubble.show()

# Sunburst chart cho phân cấp danh mục-sản phẩm
fig_sunburst = px.sunburst(
    product_summary,
    path=['Category', 'ProductName'],
    values='TotalRevenue',
    title='☀️ Phân cấp Doanh thu: Danh mục → Sản phẩm',
    color='TotalRevenue',
    color_continuous_scale='Viridis'
)

fig_sunburst.update_layout(
    height=600,
    title_x=0.5,
    title_font_size=16
)

fig_sunburst.show()

## 👥 Phân tích Khách hàng Tương tác

In [ ]:
# Tạo dữ liệu khách hàng cho phân tích
customer_summary = df.groupby(['CustomerName', 'City']).agg({
    'Revenue': 'sum',
    'OrderID': 'nunique',
    'Quantity': 'sum',
    'OrderDate': ['min', 'max']
}).reset_index()

customer_summary.columns = ['CustomerName', 'City', 'TotalRevenue', 'OrderCount', 'TotalQuantity', 'FirstOrder', 'LastOrder']
customer_summary['DaysSinceFirstOrder'] = (customer_summary['LastOrder'] - customer_summary['FirstOrder']).dt.days
customer_summary['AvgOrderValue'] = customer_summary['TotalRevenue'] / customer_summary['OrderCount']

# Treemap cho khách hàng theo thành phố
fig_treemap = px.treemap(
    customer_summary,
    path=['City', 'CustomerName'],
    values='TotalRevenue',
    title='🗺️ Bản đồ Khách hàng theo Thành phố (Size = Doanh thu)',
    color='AvgOrderValue',
    color_continuous_scale='RdYlBu',
    hover_data={'OrderCount': True, 'TotalQuantity': True}
)

fig_treemap.update_layout(
    height=600,
    title_x=0.5,
    title_font_size=16
)

fig_treemap.show()

# Scatter plot 3D cho phân tích RFM
# Tính toán Recency, Frequency, Monetary
max_date = df['OrderDate'].max()
customer_summary['Recency'] = (max_date - customer_summary['LastOrder']).dt.days
customer_summary['Frequency'] = customer_summary['OrderCount']
customer_summary['Monetary'] = customer_summary['TotalRevenue']

fig_3d = px.scatter_3d(
    customer_summary,
    x='Recency',
    y='Frequency', 
    z='Monetary',
    color='City',
    size='TotalQuantity',
    hover_name='CustomerName',
    title='🎯 Phân tích RFM 3D: Recency - Frequency - Monetary',
    labels={
        'Recency': 'Recency (ngày)',
        'Frequency': 'Frequency (số đơn)',
        'Monetary': 'Monetary ($)'
    }
)

fig_3d.update_layout(
    height=700,
    title_x=0.5,
    title_font_size=16
)

fig_3d.show()

## 📅 Phân tích Xu hướng Thời gian Tương tác

In [ ]:
# Tạo dữ liệu thời gian chi tiết
daily_stats = df.groupby('OrderDate').agg({
    'Revenue': 'sum',
    'OrderID': 'nunique',
    'CustomerID': 'nunique',
    'Quantity': 'sum'
}).reset_index()
daily_stats.columns = ['Date', 'Revenue', 'Orders', 'Customers', 'Quantity']

# Tính toán moving average
daily_stats['Revenue_MA'] = daily_stats['Revenue'].rolling(window=3, center=True).mean()
daily_stats['CumulativeRevenue'] = daily_stats['Revenue'].cumsum()

# Tạo subplot cho xu hướng thời gian
fig_time = make_subplots(
    rows=3, cols=1,
    subplot_titles=(
        '💰 Doanh thu hàng ngày và Moving Average',
        '📈 Doanh thu lũy kế theo thời gian',
        '📊 Số đơn hàng và Khách hàng theo ngày'
    ),
    vertical_spacing=0.08
)

# 1. Doanh thu hàng ngày
fig_time.add_trace(
    go.Scatter(
        x=daily_stats['Date'],
        y=daily_stats['Revenue'],
        mode='lines+markers',
        name='Doanh thu hàng ngày',
        line=dict(color='blue'),
        marker=dict(size=8)
    ),
    row=1, col=1
)

fig_time.add_trace(
    go.Scatter(
        x=daily_stats['Date'],
        y=daily_stats['Revenue_MA'],
        mode='lines',
        name='Moving Average (3 ngày)',
        line=dict(color='red', dash='dash')
    ),
    row=1, col=1
)

# 2. Doanh thu lũy kế
fig_time.add_trace(
    go.Scatter(
        x=daily_stats['Date'],
        y=daily_stats['CumulativeRevenue'],
        mode='lines',
        name='Doanh thu lũy kế',
        line=dict(color='green', width=3),
        fill='tonexty'
    ),
    row=2, col=1
)

# 3. Số đơn hàng và khách hàng
fig_time.add_trace(
    go.Bar(
        x=daily_stats['Date'],
        y=daily_stats['Orders'],
        name='Số đơn hàng',
        marker_color='orange',
        opacity=0.7
    ),
    row=3, col=1
)

fig_time.add_trace(
    go.Scatter(
        x=daily_stats['Date'],
        y=daily_stats['Customers'],
        mode='lines+markers',
        name='Số khách hàng',
        line=dict(color='purple'),
        yaxis='y2'
    ),
    row=3, col=1
)

# Cập nhật layout
fig_time.update_layout(
    height=1000,
    title_text="📅 PHÂN TÍCH XU HƯỚNG THỜI GIAN CHI TIẾT",
    title_x=0.5,
    title_font_size=18,
    showlegend=True
)

# Cập nhật axes
fig_time.update_yaxes(title_text="Doanh thu ($)", row=1, col=1)
fig_time.update_yaxes(title_text="Doanh thu lũy kế ($)", row=2, col=1)
fig_time.update_yaxes(title_text="Số đơn hàng", row=3, col=1)
fig_time.update_xaxes(title_text="Ngày", row=3, col=1)

fig_time.show()

## 🎨 Heatmap và Correlation Analysis

In [ ]:
# Tạo pivot table cho heatmap
heatmap_data = df.pivot_table(
    values='Revenue',
    index='Category',
    columns='City',
    aggfunc='sum',
    fill_value=0
)

# Heatmap doanh thu theo danh mục và thành phố
fig_heatmap = px.imshow(
    heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    color_continuous_scale='Viridis',
    title='🔥 Heatmap: Doanh thu theo Danh mục và Thành phố',
    labels={'x': 'Thành phố', 'y': 'Danh mục sản phẩm', 'color': 'Doanh thu ($)'}
)

fig_heatmap.update_layout(
    height=500,
    title_x=0.5,
    title_font_size=16
)

fig_heatmap.show()

# Correlation matrix
correlation_data = df[['Price', 'Quantity', 'Revenue']].corr()

fig_corr = px.imshow(
    correlation_data.values,
    x=correlation_data.columns,
    y=correlation_data.index,
    color_continuous_scale='RdBu',
    title='📊 Ma trận Tương quan: Giá - Số lượng - Doanh thu',
    text_auto=True
)

fig_corr.update_layout(
    height=400,
    title_x=0.5,
    title_font_size=16
)

fig_corr.show()

print("\n📈 PHÂN TÍCH TƯƠNG QUAN:")
print(f"🔗 Tương quan Giá - Số lượng: {correlation_data.loc['Price', 'Quantity']:.3f}")
print(f"🔗 Tương quan Giá - Doanh thu: {correlation_data.loc['Price', 'Revenue']:.3f}")
print(f"🔗 Tương quan Số lượng - Doanh thu: {correlation_data.loc['Quantity', 'Revenue']:.3f}")

## 💾 Xuất Dashboard và Kết thúc

In [ ]:
# Lưu các biểu đồ ra file HTML
import os
os.makedirs('/app/data/dashboards', exist_ok=True)

# Lưu dashboard chính
fig.write_html('/app/data/dashboards/main_dashboard.html')
fig_bubble.write_html('/app/data/dashboards/product_analysis.html')
fig_treemap.write_html('/app/data/dashboards/customer_analysis.html')
fig_time.write_html('/app/data/dashboards/time_analysis.html')
fig_heatmap.write_html('/app/data/dashboards/heatmap_analysis.html')

print("✅ Đã lưu tất cả dashboard vào thư mục /app/data/dashboards/")
print("📁 Các file dashboard đã tạo:")
print("   - main_dashboard.html")
print("   - product_analysis.html")
print("   - customer_analysis.html")
print("   - time_analysis.html")
print("   - heatmap_analysis.html")

# Lưu dữ liệu tổng hợp
df.to_csv('/app/data/complete_sales_data.csv', index=False)
daily_stats.to_csv('/app/data/daily_statistics.csv', index=False)
customer_summary.to_csv('/app/data/customer_summary.csv', index=False)
product_summary.to_csv('/app/data/product_summary.csv', index=False)

print("\n📊 Đã lưu dữ liệu phân tích:")
print("   - complete_sales_data.csv")
print("   - daily_statistics.csv")
print("   - customer_summary.csv")
print("   - product_summary.csv")

# Đóng kết nối
db.close()
print("\n🔌 Đã đóng kết nối database")
print("✅ Hoàn thành tạo Dashboard tương tác với Plotly!")
print("\n🎉 Bạn có thể mở các file HTML trong thư mục /app/data/dashboards/ để xem dashboard tương tác!")